# Import

In [1]:
import os
from pathlib import Path 
from typing import Optional, Literal
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms

from utils.data.dataset import CorruptedDataset
from utils.visual.visualizer import DatasetVisualizer
from utils.eval.metrics import PredictionCollector, MetricsCalculator

# FreqNorm method import
from model.method.freq_norm import LGradFreqNorm, create_lgrad_freqnorm
from model.LGrad.lgrad_model import LGrad

# GPU and Model select

In [2]:
!nvidia-smi

Sat Dec 13 16:03:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   42C    P0              34W / 250W |      4MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
DEVICE="cuda:0"
MODEL_LIST = ["lgrad", "npr"]
MODEL = MODEL_LIST[0]

# FreqNorm Configuration

In [4]:
# FreqNorm 파라미터 설정
RHO = 0.3  # Correction strength (0.5~1.0)
ALPHA_MIN = 0.85  # Minimum gain
ALPHA_MAX = 1.15  # Maximum gain
FREQ_REFERENCE = "/workspace/robust_deepfake_ai/freq_reference_progan.pth"

print(f"FreqNorm Config:")
print(f"  Rho (correction strength): {RHO}")
print(f"  Alpha min/max: {ALPHA_MIN} ~ {ALPHA_MAX}")
print(f"  Reference: {FREQ_REFERENCE}")
print(f"  Device: {DEVICE}")

FreqNorm Config:
  Rho (correction strength): 0.3
  Alpha min/max: 0.85 ~ 1.15
  Reference: /workspace/robust_deepfake_ai/freq_reference_progan.pth
  Device: cuda:0


# Dataloader

In [5]:
ROOT = "/workspace/robust_deepfake_ai/corrupted_dataset"
DATASETS = ["corrupted_test_data_biggan", "corrupted_test_data_crn", "corrupted_test_data_cyclegan", "corrupted_test_data_deepfake", "corrupted_test_data_gaugan", "corrupted_test_data_imle", "corrupted_test_data_progan", "corrupted_test_data_san", "corrupted_test_data_seeingdark", "corrupted_test_data_stargan", "corrupted_test_data_stylegan", "corrupted_test_data_stylegan2", "corrupted_test_data_whichfaceisreal"]
CORRUPTIONS = ["original", "contrast", "fog", "gaussian_noise", "jpeg_compression", "motion_blur", "pixelate"]

transform=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

In [6]:
dataset = CorruptedDataset(
    root= ROOT,
    datasets=DATASETS,
    corruptions=CORRUPTIONS,
    transform=transform
)

print(f"Total samples: {len(dataset)}")

Total samples: 632303


In [7]:
# # dataset sampling
# from torch.utils.data import Subset
# import random

# samples_per_combination = 500
# selected_indices = []

# random.seed(42)

# for dataset_name in DATASETS:
#     for corruption in CORRUPTIONS:
#         combination_indices = [
#             i for i, s in enumerate(dataset.samples)
#             if s['dataset'] == dataset_name and s['corruption'] == corruption]

#         # 1000개 샘플링 (부족하면 전부 사용)
#         n_samples = min(samples_per_combination, len(combination_indices))
#         if n_samples > 0:
#             sampled = random.sample(combination_indices, n_samples)
#             selected_indices.extend(sampled)

#         print(f"{dataset_name}-{corruption}: {len(combination_indices)} -> {n_samples} samples")

# # Subset 생성
# subset_dataset = Subset(dataset, selected_indices)
# print(f"\nTotal: {len(dataset)} -> {len(subset_dataset)} samples")

# dataloader = DataLoader(
#     subset_dataset,
#     batch_size=32,
#     shuffle=False,
#     num_workers=4,
# )

In [8]:
# dataloader = DataLoader(
#     dataset,
#     batch_size=32,
#     shuffle=False,
#     num_workers=4,
#     pin_memory=True
# )

In [9]:
# # Visualization
# viz = DatasetVisualizer(seed=1)

# viz(dataset, corruption="all", n_samples=3, label="real")

# viz.stats(dataset)

# Model load

In [10]:
# LGrad weights
STYLEGAN_WEIGHTS_ROOT = "/workspace/robust_deepfake_ai/model/LGrad/weights/karras2019stylegan-bedrooms-256x256_discriminator.pth"
CLASSIFIER_WEIGHTS_ROOT = "/workspace/robust_deepfake_ai/model/LGrad/weights/LGrad-Pretrained-Model/LGrad-4class-Trainon-Progan_car_cat_chair_horse.pth"

print("Loading LGradFreqNorm model...")
model = create_lgrad_freqnorm(
    stylegan_weights=STYLEGAN_WEIGHTS_ROOT,
    classifier_weights=CLASSIFIER_WEIGHTS_ROOT,
    reference_stats_path=FREQ_REFERENCE,
    rho=RHO,
    alpha_min=ALPHA_MIN,
    alpha_max=ALPHA_MAX,
    device=DEVICE
)
print("LGradFreqNorm ready!")

model.eval()

Loading LGradFreqNorm model...


/workspace/robust_deepfake_ai/model/LGrad/lgrad_model.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(stylegan_weights, map_location="cpu"),
/workspace/robus

LGradFreqNorm ready!


LGradFreqNorm(
  (lgrad): LGrad(
    (grad_model): StyleGANDiscriminator(
      (input0): ConvBlock(
        (mbstd): Identity()
        (blur): Identity()
        (downsample): Identity()
        (activate): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (layer0): ConvBlock(
        (mbstd): Identity()
        (blur): Identity()
        (downsample): Identity()
        (activate): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (layer1): ConvBlock(
        (mbstd): Identity()
        (blur): BlurLayer()
        (downsample): Identity()
        (activate): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (input1): ConvBlock(
        (mbstd): Identity()
        (blur): Identity()
        (downsample): Identity()
        (activate): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (layer2): ConvBlock(
        (mbstd): Identity()
        (blur): Identity()
        (downsample): Identity()
        (activate): LeakyReLU(negative_slope=0.2, inplace=True

In [11]:
from torchsummary import summary
summary(model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Identity-1          [-1, 3, 256, 256]               0
          Identity-2          [-1, 3, 256, 256]               0
          Identity-3         [-1, 64, 256, 256]               0
         LeakyReLU-4         [-1, 64, 256, 256]               0
         ConvBlock-5         [-1, 64, 256, 256]             256
          Identity-6         [-1, 64, 256, 256]               0
          Identity-7         [-1, 64, 256, 256]               0
          Identity-8         [-1, 64, 256, 256]               0
         LeakyReLU-9         [-1, 64, 256, 256]               0
        ConvBlock-10         [-1, 64, 256, 256]          36,928
         Identity-11         [-1, 64, 256, 256]               0
        BlurLayer-12         [-1, 64, 256, 256]               0
         Identity-13        [-1, 128, 128, 128]               0
        LeakyReLU-14        [-1, 128, 1

In [12]:
# def collate_fn(batch):
#     # 모델이 사용하는 resize 크기 (LGrad의 경우 256)
#     import torch
#     import torch.nn.functional as F
#     target_size = (256, 256)

#     images = []
#     for item in batch:
#         img = item[0]
#         if isinstance(img, torch.Tensor):
#             # 크기가 다르면 미리 resize (모델 내부 transform과 동일하게)
#             if img.shape[-2:] != target_size:
#                 img = F.interpolate(
#                     img.unsqueeze(0),
#                     size=target_size,
#                     mode='bilinear',
#                     align_corners=False
#                 ).squeeze(0)
#             images.append(img)
#         else:
#             images.append(img)

#     images = torch.stack(images)
#     labels = torch.tensor([item[1] for item in batch])

#     if len(batch[0]) == 3:
#         metadata = [item[2] for item in batch]
#         return images, labels, metadata
#     return images, labels

In [13]:
from torch.utils.data import Subset, DataLoader
import random

# Evaluation with LGradFreqNorm
print("="*80)
print("LGradFreqNorm Evaluation on Corrupted Datasets")
print("="*80)

calc = MetricsCalculator()
all_results = {}

for dataset_name in DATASETS:
    for corruption in CORRUPTIONS:
        # 해당 조합의 인덱스 찾기
        combination_indices = [
            i for i, s in enumerate(dataset.samples)
            if s['dataset'] == dataset_name and s['corruption'] == corruption
        ]

        if len(combination_indices) == 0:
            print(f"{dataset_name}-{corruption}: 샘플 없음, 스킵")
            continue

        print(f"\n{'='*60}")
        print(f"평가 중: {dataset_name}-{corruption}")
        print(f"샘플 수: {len(combination_indices)}")
        print(f"{'='*60}")

        # Subset과 DataLoader 생성
        subset = Subset(dataset, combination_indices)
        dataloader = DataLoader(
            subset,
            batch_size=16,
            shuffle=False,
            num_workers=4,
            drop_last=True
        )

        # 평가
        metrics = calc.evaluate(
            model=model,
            dataloader=dataloader,
            device=DEVICE,
            name=f"{dataset_name}-{corruption}"
        )

        # 즉시 결과 출력
        print(f"\n[LGradFreqNorm] 결과:")
        print(f"  Accuracy: {metrics['accuracy']*100:.2f}%")
        print(f"  AUC:      {metrics['auc']*100:.2f}%")
        print(f"  AP:       {metrics['ap']*100:.2f}%")
        print(f"  F1:       {metrics['f1']*100:.2f}%")

        # 결과 저장
        all_results[(dataset_name, corruption)] = metrics

# 전체 결과 테이블 출력
print(f"\n\n{'='*80}")
print("LGradFreqNorm - 전체 결과 요약")
print(f"{'='*80}\n")
calc.print_results_table()
calc.summarize_by_corruption(all_results)
calc.summarize_by_dataset(all_results)

LGradFreqNorm Evaluation on Corrupted Datasets

평가 중: corrupted_test_data_biggan-original
샘플 수: 4000


corrupted_test_data_biggan-original: 100%|██████████| 250/250 [00:47<00:00,  5.24it/s]



[LGradFreqNorm] 결과:
  Accuracy: 78.53%
  AUC:      89.75%
  AP:       87.21%
  F1:       81.60%

평가 중: corrupted_test_data_biggan-contrast
샘플 수: 4000


corrupted_test_data_biggan-contrast: 100%|██████████| 250/250 [00:47<00:00,  5.26it/s]



[LGradFreqNorm] 결과:
  Accuracy: 58.85%
  AUC:      64.30%
  AP:       58.92%
  F1:       70.42%

평가 중: corrupted_test_data_biggan-fog
샘플 수: 4000


...

corrupted_test_data_whichfaceisreal-motion_blur: 100%|██████████| 125/125 [00:24<00:00,  5.04it/s]



[LGradFreqNorm] 결과:
  Accuracy: 51.80%
  AUC:      66.32%
  AP:       60.81%
  F1:       67.43%

평가 중: corrupted_test_data_whichfaceisreal-pixelate
샘플 수: 2000


corrupted_test_data_whichfaceisreal-pixelate: 100%|██████████| 125/125 [00:24<00:00,  5.07it/s]



[LGradFreqNorm] 결과:
  Accuracy: 61.05%
  AUC:      68.53%
  AP:       65.69%
  F1:       69.96%


LGradFreqNorm - 전체 결과 요약



ModuleNotFoundError: No module named 'rich'

In [ ]:
# # Evaluation
# calc = MetricsCalculator()

# # 조합별 평가
# results = calc.evaluate(
#     model=model,
#     dataloader=dataloader,
#     device=DEVICE,
#     name=f"{dataset_name}-{corruption}"
# )

# # 결과 출력
# calc.print_results_table(results)
# calc.summarize_by_corruption(results)
# calc.summarize_by_dataset(results)
